In [1]:
import pandas as pd
from gensim.models import Word2Vec
import numpy as np
import feather
import keras
from keras.layers import Dense
from keras.models import  Sequential
from sklearn.metrics import accuracy_score 
from sklearn.model_selection import train_test_split
import os
import tensorflow as tf
from keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(monitor='val_loss', patience=2, verbose=2)

Using TensorFlow backend.


In [2]:
path_train = './data/train_preliminary/'
path_test = './data/test/'

In [3]:
train_ad = pd.read_csv(path_train + "ad.csv")
train_user = pd.read_csv(path_train + 'user.csv')
train_click_log = pd.read_csv(path_train + 'click_log.csv')

In [4]:
test_ad = pd.read_csv(path_test + "ad.csv")
test_click_log = pd.read_csv(path_test + 'click_log.csv')

In [5]:
for column in train_ad.columns:
    print(train_ad[column].value_counts())

2047       1
3474441    1
3505158    1
3494915    1
3492864    1
701439     1
693243     1
695290     1
4406371    1
719862     1
713717     1
709619     1
705521     1
668655     1
670702     1
664557     1
666604     1
3503111    1
3484684    1
687078     1
3482637    1
3410984    1
3439654    1
3433509    1
3435556    1
3429411    1
3431458    1
3515421    1
3517468    1
3511323    1
          ..
2122072    1
2103639    1
2101590    1
2099541    1
2107729    1
2152783    1
2148685    1
2146636    1
2158922    1
2156873    1
2154824    1
2117982    1
2120031    1
2204000    1
2171248    1
2167158    1
2165109    1
4322370    1
2177395    1
2175346    1
2173297    1
2226539    1
2206049    1
2224490    1
2220392    1
2201959    1
2197861    1
2195812    1
2210147    1
4194304    1
Name: creative_id, Length: 2481135, dtype: int64
1729961    19
2179954    18
1771929    17
2763747    17
3718339    16
681564     16
847568     16
926023     16
3288857    16
1779604    16
2475922    16
3461

In [6]:
train_ad

,creative_id,ad_id,product_id,product_category,advertiser_id,industry
0,1,1,\N,5,381,78
1,4,4,\N,5,108,202
2,7,7,\N,5,148,297
3,8,8,\N,5,713,213
4,9,9,\N,5,695,213
5,10,10,\N,5,100,73
6,12,12,\N,5,765,6
7,13,13,\N,5,113,267
8,16,16,\N,5,623,1
9,20,20,34647,5,312,267


In [7]:
for column in train_click_log.columns:
    print(train_click_log[column].value_counts())

85    417797
78    406004
84    393253
64    388689
57    384115
74    381696
91    376422
63    376263
34    373768
36    371639
31    369978
77    368169
73    367376
43    363130
35    362195
75    360971
65    360797
37    359112
32    358182
33    357740
71    357655
56    357324
79    355733
67    354459
50    353179
70    350897
66    349208
88    349030
90    348829
81    348312
       ...  
15    317617
29    316204
44    314554
13    312896
39    310426
21    309926
18    307755
28    307321
38    305780
1     298107
26    293122
17    293056
8     291980
19    291491
16    289072
25    288726
11    288523
7     287337
27    286883
23    286319
2     284214
24    283650
12    282593
10    275954
3     273636
20    273404
4     271385
5     268042
9     259223
6     256799
Name: time, Length: 91, dtype: int64
839368    16868
157585     1706
677301     1661
178333     1321
549175     1273
524400     1237
98218      1225
280513     1162
674070     1160
407744     1011
16962     

In [8]:
train_click_log

,time,user_id,creative_id,click_times
0,9,30920,567330,1
1,65,30920,3072255,1
2,56,30920,2361327,1
3,6,309204,325532,1
4,59,309204,2746730,1
5,12,309204,726402,1
6,79,309204,2851451,1
7,32,309204,1569716,1
8,5,309204,71956,1
9,8,309204,322354,1


In [9]:
train_user

,user_id,age,gender
0,1,4,1
1,2,10,1
2,3,7,2
3,4,5,1
4,5,4,1
5,6,6,1
6,7,6,2
7,8,5,1
8,9,5,1
9,10,9,2


In [10]:
train_tmp = pd.merge(train_ad, train_click_log, how='left', on='creative_id')
train_total = pd.merge(train_user, train_tmp, how='left', on='user_id')
train_total
del train_tmp

In [11]:
test_total = pd.merge(test_click_log, test_ad,  how='left', on='creative_id')
test_total['age']=0
test_total['gender']=0
test_total

,time,user_id,creative_id,click_times,ad_id,product_id,product_category,advertiser_id,industry,age,gender
0,20,3131989,645764,1,573314,58,2,14689,6,0,0
1,20,3131989,1027422,1,902764,129,2,42272,6,0,0
2,20,3131989,1106443,1,970829,2171,2,37513,322,0,0
3,20,3131989,629802,1,559183,\N,18,14678,26,0,0
4,59,3131989,2839769,1,2441288,129,2,35328,6,0,0
5,16,3131989,564033,1,501478,129,2,38382,6,0,0
6,17,3131989,103304,1,93874,\N,18,14,23,0,0
7,11,3131989,577903,1,513975,2171,2,37513,322,0,0
8,20,3131989,103304,1,93874,\N,18,14,23,0,0
9,16,3131989,103304,1,93874,\N,18,14,23,0,0


In [12]:
data_total = pd.concat([train_total,test_total], axis=0)
data_total

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


,ad_id,advertiser_id,age,click_times,creative_id,gender,industry,product_category,product_id,time,user_id
0,66210,14681,4,1,71691,1,326,18,\N,43,1
1,82421,367,4,1,90171,1,217,5,\N,52,1
2,109959,11411,4,1,122032,1,\N,2,1334,40,1
3,188507,9702,4,1,209778,1,6,2,136,20,1
4,724607,7293,4,1,821396,1,326,5,\N,20,1
5,773445,29455,4,1,877468,1,106,5,\N,20,1
6,1458878,14668,4,1,1683713,1,326,5,\N,39,1
7,1678901,17189,4,1,1940159,1,73,5,\N,46,1
8,1803758,44865,4,1,2087846,1,64,18,\N,60,1
9,2116146,188,4,1,2456345,1,245,2,64,73,1


In [13]:
data_total['product_id'].replace('\\N', 0, inplace=True) 
data_total['industry'].replace('\\N', 0, inplace=True) 

In [14]:
data_total = data_total.sort_values(by='time', ascending=True)

In [15]:
data_total

,ad_id,advertiser_id,age,click_times,creative_id,gender,industry,product_category,product_id,time,user_id
2622755,192043,14681,0,1,213611,0,88,18,0,1,3287148
20116123,134888,25051,8,1,150354,1,277,8,11399,1,602069
7677871,214981,26469,0,1,239250,0,247,13,2961,1,3589355
16642067,154359,10985,0,2,172021,0,6,2,1261,1,3117482
16642063,140636,7115,0,1,156869,0,6,2,1261,1,3117482
7677985,62852,918,0,1,67978,0,6,2,1261,1,3589365
13147059,203159,16327,2,1,226367,1,247,2,2200,1,392972
20481018,29748,11990,0,1,29762,0,321,2,1234,1,3347747
7677994,97286,24475,0,1,107103,0,250,2,2168,1,3589365
20481046,147502,27404,0,1,164538,0,247,3,9127,1,3347748


In [16]:
# data_total.groupby('product_category').value_counts.unstack.rest_index
data_total.groupby(['age', 'product_category'])['click_times'].sum().sort_values(ascending = False)

age  product_category
0    2                   15025357
     18                  10197381
     3                    3403376
3    2                    2931356
0    5                    2761221
2    2                    2417421
4    2                    2135129
3    18                   2047779
5    2                    1865272
0    12                   1743335
2    18                   1652782
4    18                   1451960
6    2                    1402368
5    18                   1230822
6    18                    969043
0    8                     944908
7    2                     902909
0    17                    867023
1    2                     777696
3    3                     760783
7    18                    664588
4    3                     564745
2    3                     557746
3    5                     557014
5    3                     458571
0    13                    456944
8    2                     448403
4    5                     423009
1    18                   

In [16]:
data_total['creative_id'] = data_total['creative_id'].astype('str')
data_total['user_id'] = data_total['user_id'].astype('str')
data_total['ad_id'] = data_total['ad_id'].astype('str')
data_total['advertiser_id'] = data_total['advertiser_id'].astype('str')
data_total['industry'] = data_total['industry'].astype('str')
data_total['product_category'] = data_total['product_category'].astype('str')
data_total['product_id'] = data_total['product_id'].astype('str')
data_total['time'] = data_total['time'].astype('str')
data_total['click_times'] = data_total['click_times'].astype('str')

data_total= data_total[['user_id', 'age', 'gender','creative_id','ad_id', 'advertiser_id', 'industry','product_category', 'product_id', 'time', 'click_times']]
data_total

,user_id,age,gender,creative_id,ad_id,advertiser_id,industry,product_category,product_id,time,click_times
2622755,3287148,0,0,213611,192043,14681,88,18,0,1,1
20116123,602069,8,1,150354,134888,25051,277,8,11399,1,1
7677871,3589355,0,0,239250,214981,26469,247,13,2961,1,1
16642067,3117482,0,0,172021,154359,10985,6,2,1261,1,2
16642063,3117482,0,0,156869,140636,7115,6,2,1261,1,1
7677985,3589365,0,0,67978,62852,918,6,2,1261,1,1
13147059,392972,2,1,226367,203159,16327,247,2,2200,1,1
20481018,3347747,0,0,29762,29748,11990,321,2,1234,1,1
7677994,3589365,0,0,107103,97286,24475,250,2,2168,1,1
20481046,3347748,0,0,164538,147502,27404,247,3,9127,1,1


In [17]:
user_num = data_total['user_id'].unique()
user_num

array(['3287148', '602069', '3589355', ..., '868400', '3819235',
       '3083446'], dtype=object)

In [18]:
del train_total, test_total

In [20]:
# models = [None for i in range(8)]
# grouped = data_total.groupby('user_id')
# # word_lib = 2481135
# w2v_list = ['creative_id','ad_id', 'advertiser_id', 'industry','product_category', 'product_id', 'time', 'click_times']
# for i, feature in enumerate(w2v_list): 
#     print(feature)
#     voca_lib = []
#     for j, user in enumerate(user_num):
#         if j%100000==0:
#             print(j)
#         voca_lib.append(list(grouped.get_group(user)[feature]))
#     model = Word2Vec(voca_lib, size=100, min_count=1)
#     models[i] = model
#     model.save('./model/w2v_sort/w2v'+feature+'_.model')

creative_id
0
100000
200000
300000
400000
500000
600000
700000
800000
900000
1000000
1100000
1200000
1300000
1400000
1500000
1600000
1700000
1800000
ad_id
0
100000
200000
300000
400000
500000
600000
700000
800000
900000
1000000
1100000
1200000
1300000
1400000
1500000
1600000
1700000
1800000
advertiser_id
0
100000
200000
300000
400000
500000
600000
700000
800000
900000
1000000
1100000
1200000
1300000
1400000
1500000
1600000
1700000
1800000
industry
0
100000
200000
300000
400000
500000
600000
700000
800000
900000
1000000
1100000
1200000
1300000
1400000
1500000
1600000
1700000
1800000
product_category
0
100000
200000
300000
400000
500000
600000
700000
800000
900000
1000000
1100000
1200000
1300000
1400000
1500000
1600000
1700000
1800000
product_id
0
100000
200000
300000
400000
500000
600000
700000
800000
900000
1000000
1100000
1200000
1300000
1400000
1500000
1600000
1700000
1800000
time
0
100000
200000
300000
400000
500000
600000
700000
800000
900000
1000000
1100000
1200000
1300000
1400000

In [19]:
df_total = data_total.drop_duplicates(subset='user_id', keep='first')[['user_id', 'gender','age']]
df_total

,user_id,gender,age
2622755,3287148,0,0
20116123,602069,1,8
7677871,3589355,0,0
16642067,3117482,0,0
7677985,3589365,0,0
13147059,392972,1,2
20481018,3347747,0,0
20481046,3347748,0,0
13147089,392973,1,3
5340764,159308,2,1


In [20]:
tmp = data_total.groupby('user_id')['creative_id','ad_id', 'advertiser_id', 'industry','product_category', 'product_id', 'time', 'click_times'].agg(lambda x:list(x))
tmp

,creative_id,ad_id,advertiser_id,industry,product_category,product_id,time,click_times
user_id,,,,,,,,
1,"[821396, 209778, 877468, 1683713, 122032, 7169...","[724607, 188507, 773445, 1458878, 109959, 6621...","[7293, 9702, 29455, 14668, 11411, 14681, 17189...","[326, 6, 106, 326, 0, 326, 73, 217, 64, 245, 2...","[5, 2, 5, 5, 2, 18, 5, 5, 18, 2, 2, 2, 2]","[0, 136, 0, 0, 1334, 0, 0, 0, 0, 64, 1454, 64,...","[20, 20, 20, 39, 40, 43, 46, 52, 60, 64, 64, 7...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2]"
10,"[134938, 69204, 197464, 81006, 3331781, 71689,...","[121278, 63976, 177187, 74359, 2866446, 66208,...","[62956, 24952, 2367, 953, 52604, 14681, 14681,...","[60, 60, 60, 158, 6, 6, 6, 318, 231, 231]","[3, 3, 3, 18, 2, 18, 18, 2, 2, 2]","[26858, 26858, 26858, 0, 136, 0, 0, 87, 161, 161]","[4, 7, 22, 28, 70, 73, 74, 79, 84, 91]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]"
100,"[417690, 69319, 203686, 882344, 1130294, 13637...","[372742, 64080, 182889, 777688, 990980, 119157...","[2003, 30825, 5883, 10989, 2003, 22716, 39177,...","[40, 322, 40, 297, 40, 99, 322, 238, 247, 322,...","[18, 18, 18, 18, 18, 5, 18, 18, 3, 18, 3, 18, ...","[0, 0, 0, 0, 0, 0, 0, 0, 8828, 0, 8988, 0, 0, ...","[5, 5, 10, 23, 23, 25, 26, 29, 31, 31, 35, 40,...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
1000,"[246393, 246134, 556859, 246122, 219265, 61805...","[221217, 220899, 495498, 220883, 196846, 54878...","[21053, 21053, 16046, 21053, 19051, 27153, 185...","[247, 247, 247, 247, 242, 319, 6, 88, 6, 205, ...","[13, 13, 13, 13, 2, 2, 2, 18, 18, 18, 18, 2, 1...","[2487, 2487, 2449, 2487, 111, 1296, 129, 0, 0,...","[2, 8, 8, 8, 14, 17, 23, 24, 24, 24, 24, 24, 2...","[1, 1, 1, 1, 1, 1, 1, 3, 1, 1, 1, 1, 1, 1, 1, ..."
10000,"[71633, 210949, 481943, 84849, 283062, 24333, ...","[66158, 189560, 428649, 77709, 254705, 24966, ...","[29966, 7273, 19771, 22020, 7212, 22885, 16333...","[322, 322, 165, 252, 322, 318, 261, 321, 321, ...","[2, 18, 5, 2, 18, 2, 2, 2, 2, 2, 2, 2, 2, 18, ...","[1243, 0, 0, 2151, 0, 87, 1356, 1075, 1075, 18...","[7, 8, 9, 10, 11, 12, 13, 15, 15, 15, 15, 21, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
100000,"[1052523, 764756, 875021, 32795, 71688, 39714,...","[924114, 675869, 771243, 32161, 66207, 38066, ...","[14142, 817, 23746, 18563, 14681, 18562, 1132,...","[317, 317, 6, 6, 6, 6, 319, 154, 47, 54, 161, ...","[2, 2, 2, 2, 18, 2, 2, 2, 18, 18, 2, 2, 2, 2, 2]","[1593, 29, 129, 129, 0, 129, 1370, 1164, 0, 0,...","[19, 19, 23, 23, 23, 25, 31, 33, 38, 41, 58, 6...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]"
100001,"[637949, 291979, 116004, 896361, 1441710, 2142...","[566554, 262385, 104842, 790202, 1256663, 1850...","[25378, 14681, 31031, 39252, 46279, 26006, 422...","[238, 6, 319, 242, 238, 47, 6, 319, 322, 47, 4...","[2, 18, 2, 2, 2, 18, 2, 2, 2, 18, 18, 2, 18, 18]","[2031, 0, 1370, 1064, 1380, 0, 129, 1370, 1873...","[21, 21, 21, 28, 40, 47, 47, 47, 51, 64, 67, 6...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]"
100002,"[217635, 328796, 3072233, 3191379, 3183339, 32...","[195447, 295469, 2642282, 2745652, 2738765, 28...","[22127, 6714, 6783, 36976, 37513, 57597, 57597...","[6, 5, 6, 319, 322, 317, 317, 317, 319, 317, 3...","[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 18, 18, 1...","[129, 1268, 1261, 1810, 2171, 1849, 1849, 1849...","[8, 23, 69, 70, 71, 74, 76, 77, 79, 79, 79, 81...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, ..."
100003,"[235830, 239657, 77859, 88503, 108155, 1029025...","[212070, 215330, 71677, 81004, 98183, 904179, ...","[23701, 18767, 22237, 10989, 10989, 10986, 176...","[73, 21, 54, 326, 291, 6, 54, 6, 291, 54, 203,...","[12, 18, 12, 12, 12, 12, 18, 12, 12, 12, 12, 1...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 126...","[1, 2, 4, 7, 16, 23, 30, 31, 37, 39, 41, 50, 5...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."


In [21]:
tmp = pd.DataFrame(tmp)
tmp

,creative_id,ad_id,advertiser_id,industry,product_category,product_id,time,click_times
user_id,,,,,,,,
1,"[821396, 209778, 877468, 1683713, 122032, 7169...","[724607, 188507, 773445, 1458878, 109959, 6621...","[7293, 9702, 29455, 14668, 11411, 14681, 17189...","[326, 6, 106, 326, 0, 326, 73, 217, 64, 245, 2...","[5, 2, 5, 5, 2, 18, 5, 5, 18, 2, 2, 2, 2]","[0, 136, 0, 0, 1334, 0, 0, 0, 0, 64, 1454, 64,...","[20, 20, 20, 39, 40, 43, 46, 52, 60, 64, 64, 7...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2]"
10,"[134938, 69204, 197464, 81006, 3331781, 71689,...","[121278, 63976, 177187, 74359, 2866446, 66208,...","[62956, 24952, 2367, 953, 52604, 14681, 14681,...","[60, 60, 60, 158, 6, 6, 6, 318, 231, 231]","[3, 3, 3, 18, 2, 18, 18, 2, 2, 2]","[26858, 26858, 26858, 0, 136, 0, 0, 87, 161, 161]","[4, 7, 22, 28, 70, 73, 74, 79, 84, 91]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]"
100,"[417690, 69319, 203686, 882344, 1130294, 13637...","[372742, 64080, 182889, 777688, 990980, 119157...","[2003, 30825, 5883, 10989, 2003, 22716, 39177,...","[40, 322, 40, 297, 40, 99, 322, 238, 247, 322,...","[18, 18, 18, 18, 18, 5, 18, 18, 3, 18, 3, 18, ...","[0, 0, 0, 0, 0, 0, 0, 0, 8828, 0, 8988, 0, 0, ...","[5, 5, 10, 23, 23, 25, 26, 29, 31, 31, 35, 40,...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
1000,"[246393, 246134, 556859, 246122, 219265, 61805...","[221217, 220899, 495498, 220883, 196846, 54878...","[21053, 21053, 16046, 21053, 19051, 27153, 185...","[247, 247, 247, 247, 242, 319, 6, 88, 6, 205, ...","[13, 13, 13, 13, 2, 2, 2, 18, 18, 18, 18, 2, 1...","[2487, 2487, 2449, 2487, 111, 1296, 129, 0, 0,...","[2, 8, 8, 8, 14, 17, 23, 24, 24, 24, 24, 24, 2...","[1, 1, 1, 1, 1, 1, 1, 3, 1, 1, 1, 1, 1, 1, 1, ..."
10000,"[71633, 210949, 481943, 84849, 283062, 24333, ...","[66158, 189560, 428649, 77709, 254705, 24966, ...","[29966, 7273, 19771, 22020, 7212, 22885, 16333...","[322, 322, 165, 252, 322, 318, 261, 321, 321, ...","[2, 18, 5, 2, 18, 2, 2, 2, 2, 2, 2, 2, 2, 18, ...","[1243, 0, 0, 2151, 0, 87, 1356, 1075, 1075, 18...","[7, 8, 9, 10, 11, 12, 13, 15, 15, 15, 15, 21, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
100000,"[1052523, 764756, 875021, 32795, 71688, 39714,...","[924114, 675869, 771243, 32161, 66207, 38066, ...","[14142, 817, 23746, 18563, 14681, 18562, 1132,...","[317, 317, 6, 6, 6, 6, 319, 154, 47, 54, 161, ...","[2, 2, 2, 2, 18, 2, 2, 2, 18, 18, 2, 2, 2, 2, 2]","[1593, 29, 129, 129, 0, 129, 1370, 1164, 0, 0,...","[19, 19, 23, 23, 23, 25, 31, 33, 38, 41, 58, 6...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]"
100001,"[637949, 291979, 116004, 896361, 1441710, 2142...","[566554, 262385, 104842, 790202, 1256663, 1850...","[25378, 14681, 31031, 39252, 46279, 26006, 422...","[238, 6, 319, 242, 238, 47, 6, 319, 322, 47, 4...","[2, 18, 2, 2, 2, 18, 2, 2, 2, 18, 18, 2, 18, 18]","[2031, 0, 1370, 1064, 1380, 0, 129, 1370, 1873...","[21, 21, 21, 28, 40, 47, 47, 47, 51, 64, 67, 6...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]"
100002,"[217635, 328796, 3072233, 3191379, 3183339, 32...","[195447, 295469, 2642282, 2745652, 2738765, 28...","[22127, 6714, 6783, 36976, 37513, 57597, 57597...","[6, 5, 6, 319, 322, 317, 317, 317, 319, 317, 3...","[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 18, 18, 1...","[129, 1268, 1261, 1810, 2171, 1849, 1849, 1849...","[8, 23, 69, 70, 71, 74, 76, 77, 79, 79, 79, 81...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, ..."
100003,"[235830, 239657, 77859, 88503, 108155, 1029025...","[212070, 215330, 71677, 81004, 98183, 904179, ...","[23701, 18767, 22237, 10989, 10989, 10986, 176...","[73, 21, 54, 326, 291, 6, 54, 6, 291, 54, 203,...","[12, 18, 12, 12, 12, 12, 18, 12, 12, 12, 12, 1...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 126...","[1, 2, 4, 7, 16, 23, 30, 31, 37, 39, 41, 50, 5...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."


In [23]:
from gensim.corpora.dictionary import Dictionary

def create_dictionaries(p_model):
    gensim_dict = Dictionary()
    gensim_dict.doc2bow(p_model.wv.vocab.keys(), allow_update=True)
    w2indx = {v: k + 1 for k, v in gensim_dict.items()}  # 词语的索引，从1开始编号
    w2vec = {word: model[word] for word in w2indx.keys()}  # 词语的词向量
    return w2indx, w2vec

In [24]:
def get_vec(arr, wv_vec, words):
    vec = np.zeros((len(arr), wv_vec.shape[1]))
    sorter = np.argsort(words)
    for i in range(len(arr)):
        try:
            a = eval(arr[i])
        except:
            print(arr[i])
        if len(a) > 0:
            index = sorter[np.searchsorted(words, a, sorter=sorter)]
            vec[i, :] = np.mean(wv_vec[index], axis=0)
    return vec

In [ ]:
### 生成各个特征的vector,并汇总于df_total
w2v_list = ['creative_id','ad_id', 'advertiser_id', 'industry','product_category', 'product_id',  'time', 'click_times']
for i, feature in enumerate(w2v_list):  
    print(feature)
    model = Word2Vec.load('./model/w2v_sort/w2v'+feature+'_.model')#('./model/w2v/w2v'+feature+'_.model')
#     model = models[i]
    print('loaded... ')
    w2indx, w2vec = create_dictionaries(model)
#     print(w2indx,w2vec)
    w2vec = pd.DataFrame(w2vec)
    w2vec = pd.DataFrame(w2vec.values.T, index=w2vec.columns, columns=w2vec.index)
    wv_vec = np.array(w2vec)
    words = np.array(w2vec.index)
    vec = get_vec(np.array(tmp[feature].astype(str)), wv_vec, words)
    print('vec calculated')
    vec = pd.DataFrame(vec)
    columns =[str(i) for i in vec.columns] #add feature name to columns 
    columns = [feature+ i for i in columns]
    vec.columns = columns
    vec['user_id'] = list(tmp.index)
    df_total = pd.merge(df_total, vec, how='left', on='user_id')
df_total

creative_id
loaded... 


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  import sys


In [ ]:
# df_total.to_feather('./tmp_data/df_total.feather')#保存df_total
df_total.to_feather('./tmp_data/df_total2.feather')#保存df_total

In [ ]:
# w2vec = pd.DataFrame(w2vec)
# w2vec

In [181]:
# w2vec = pd.DataFrame(w2vec.values.T, index=w2vec.columns, columns=w2vec.index)
# w2vec

,0,1,2,3,4,5,6,7,8,9,...,290,291,292,293,294,295,296,297,298,299
1,-0.048629,-0.096933,-0.077104,0.059804,-0.069945,0.033863,0.002142,0.029912,0.038396,0.010039,...,0.143288,0.006912,-0.009410,0.036826,-0.019614,-0.068730,-0.071941,0.004368,-0.022422,0.052838
10,0.021202,-0.000030,-0.006865,0.010586,-0.011180,0.023549,-0.006076,0.015641,0.012421,0.019546,...,0.014990,0.039793,0.000627,0.022063,0.009913,0.026309,0.004113,0.038788,-0.019799,-0.001249
100,0.013346,-0.029541,-0.018252,0.003071,-0.059532,0.017944,0.029065,0.031189,0.009929,0.017876,...,0.050048,0.005092,-0.019647,-0.003106,-0.044006,-0.013825,-0.006788,-0.040198,0.008216,0.023971
10000,0.002428,0.014105,0.005596,-0.000343,0.008237,0.002565,0.005935,-0.007755,-0.009070,0.001906,...,-0.004247,-0.002660,0.003102,-0.001442,0.009298,0.009297,-0.007263,0.005025,-0.005184,0.005612
100000,0.005381,-0.017195,0.009090,0.019389,-0.024999,-0.006924,0.013326,0.000179,0.001604,0.022345,...,0.037628,-0.019728,0.020382,-0.008598,0.020552,-0.019860,-0.011678,-0.004936,0.018573,0.011773
1000000,0.002290,-0.036547,-0.013598,0.031711,-0.032192,0.015357,0.002933,0.003728,0.005086,0.004876,...,0.040135,0.009699,-0.015737,0.021920,-0.009570,-0.003591,-0.012601,0.005287,-0.000291,0.030805
1000001,0.000346,-0.027499,0.002022,0.003607,-0.018892,0.002741,0.006600,-0.004667,-0.005566,0.004448,...,0.021053,-0.011353,-0.007949,-0.006184,-0.010588,0.000703,-0.008872,-0.006835,0.006987,0.001616
1000003,0.000222,-0.064595,-0.026943,0.023276,-0.049109,0.013783,0.004291,-0.011668,-0.013958,0.019357,...,0.061174,-0.015632,-0.015018,0.017548,-0.036757,-0.025682,-0.017871,-0.027708,0.011532,0.029722
1000004,-0.001937,-0.022337,-0.004309,0.008842,-0.020999,0.007556,-0.001086,0.017280,0.022595,-0.007758,...,0.033025,0.006425,-0.000870,-0.003612,-0.025180,-0.008539,-0.016438,-0.003570,-0.012164,-0.003741
1000006,-0.007587,-0.013711,0.016044,0.014255,0.025548,-0.004573,-0.033313,-0.032698,-0.019211,0.021908,...,0.039923,0.042689,0.002849,0.053104,-0.032831,0.018529,-0.008983,-0.012552,0.035154,0.032976


In [182]:
# words = np.array(w2vec.index)
# words

array(['1', '10', '100', ..., '999996', '999997', '999998'], dtype=object)

In [183]:
# wv_vec = np.array(w2vec)

In [185]:
# vec = get_vec(np.array(tmp["creative_id"].astype(str)), wv_vec, words)
# vec

array([[-0.29758212, -0.55405974,  0.0831165 , ..., -0.35705727,
        -0.10849164,  0.12569807],
       [-0.74726838,  0.37408876, -0.08397363, ..., -0.26534694,
        -0.97462308,  0.21798539],
       [ 0.03903237, -0.41071251, -0.27458385, ...,  0.04264095,
        -0.00708319, -0.1221389 ],
       ...,
       [-0.17242217, -1.06172585, -0.14772433, ...,  0.90219712,
        -0.47850239,  1.16035903],
       [-0.32791451, -0.64742774, -0.14896993, ...,  0.05314554,
        -0.19770978,  0.6701023 ],
       [-0.03787553, -0.57480091,  0.28065893, ..., -0.43637556,
        -0.0757186 ,  0.20872588]])

In [186]:
# tmp.index

Index(['1', '10', '100', '1000', '10000', '100000', '100001', '100002',
       '100003', '100004',
       ...
       '99990', '99991', '99992', '99993', '99994', '99995', '99996', '99997',
       '99998', '99999'],
      dtype='object', name='user_id', length=1900000)

In [187]:
# vec = pd.DataFrame(vec)
# vec['user_id'] = list(tmp.index)
# vec

,0,1,2,3,4,5,6,7,8,9,...,291,292,293,294,295,296,297,298,299,user_id
0,-0.297582,-0.554060,0.083117,0.782288,-0.069381,-0.121675,-0.365755,-0.176169,-0.361323,0.275759,...,-0.253531,0.079110,0.181841,0.003379,-0.331067,-0.491239,-0.357057,-0.108492,0.125698,1
1,-0.747268,0.374089,-0.083974,0.947993,-0.419260,0.064127,-0.146026,0.141464,-0.131365,-0.227129,...,0.393436,0.045845,-0.308215,-0.373806,0.235226,-0.261167,-0.265347,-0.974623,0.217985,10
2,0.039032,-0.410713,-0.274584,0.172928,-0.303484,0.081287,0.122858,0.632145,0.348930,-0.092921,...,-0.436870,0.123794,-0.099139,-0.343372,0.141237,-0.207223,0.042641,-0.007083,-0.122139,100
3,-0.079369,0.102116,-0.214423,0.239787,-0.041988,-0.067240,0.056159,-0.599444,-0.121784,0.387610,...,-0.218212,-0.108966,0.241433,-0.137637,0.156599,-0.405296,0.145356,-0.128270,0.155950,1000
4,0.023595,-0.350858,-0.159057,0.608887,-0.310834,0.025367,0.111958,-0.342109,-0.509201,0.477224,...,-0.437493,0.222213,0.378949,-0.238799,-0.079425,-0.261434,-0.093951,0.048136,0.452885,10000
5,0.284601,-0.521026,-0.024157,0.505256,0.163967,-0.088245,-0.403059,-0.680771,-0.517391,0.665885,...,-0.413347,0.060198,0.291787,0.176635,0.277464,-0.327381,-0.147389,0.154938,0.995884,100000
6,0.014660,-0.973780,0.157408,0.785193,0.243382,-0.372145,0.054594,-0.849905,-0.439920,0.537301,...,-0.828145,-0.109669,0.401353,0.016102,-0.116934,-0.130340,-0.118462,-0.099654,1.024121,100001
7,-0.042761,-0.658015,0.021353,0.281689,-0.315177,0.144597,0.230312,0.155306,0.369993,0.092495,...,0.211733,0.066332,-0.076460,0.010944,-0.281916,-0.239030,0.083263,-0.359875,0.553212,100002
8,0.018092,-0.746214,-0.216478,0.134908,-0.137278,0.036927,-0.057445,0.355879,0.317739,0.304228,...,-0.030146,-0.148956,0.104147,0.030900,-0.699308,-0.253842,-0.581212,-0.010972,0.474061,100003
9,0.141425,-1.015632,-0.186851,0.336824,0.048936,-0.277493,0.148360,0.063837,-0.041381,-0.168018,...,-0.127717,0.172675,0.529297,-0.049117,0.023676,-0.416550,0.320509,0.059067,0.731519,100004


In [188]:
# vec.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1900000 entries, 0 to 1899999
Columns: 301 entries, 0 to user_id
dtypes: float64(300), object(1)
memory usage: 4.3+ GB


In [189]:
# df_total = pd.merge(df_total, vec, how='left', on='user_id')
# df_total

,user_id,gender,age,0,1,2,3,4,5,6,...,290,291,292,293,294,295,296,297,298,299
0,1,1,4,-0.297582,-0.554060,0.083117,0.782288,-0.069381,-0.121675,-0.365755,...,0.372246,-0.253531,0.079110,0.181841,0.003379,-0.331067,-0.491239,-0.357057,-0.108492,0.125698
1,2,1,10,0.157778,-0.784552,-0.236854,0.364366,0.157558,-0.095884,0.024874,...,0.148401,-0.276131,-0.104864,0.364842,0.024070,0.160327,-0.391514,-0.207197,0.139020,1.075754
2,3,2,7,0.052649,-0.996827,0.098026,0.004296,-0.053774,-0.085781,-0.127117,...,0.890865,-0.101124,-0.445202,0.316014,-0.157522,-0.394284,-0.256539,-0.143678,0.084161,0.319744
3,4,1,5,-0.222638,-0.649482,-0.061883,0.040237,-0.129986,-0.235657,-0.385574,...,0.448275,0.336522,-0.272371,0.278289,-0.211749,-0.260152,-0.566912,0.033142,-0.154367,0.661096
4,5,1,4,-0.292408,-0.361831,-0.278923,0.463791,-0.457496,-0.054648,-0.071344,...,0.366558,-0.019216,-0.061525,-0.108647,-0.399550,-0.184600,-0.306554,-0.281219,-0.117435,0.247664
5,6,1,6,-0.381112,-0.812110,-0.153805,0.010151,-0.316271,-0.202187,-0.177186,...,0.456889,-0.551528,-0.216920,-0.390985,0.011132,0.106577,-0.248599,-0.489717,0.327435,-0.149077
6,7,2,6,0.228182,-0.580980,0.135177,0.312805,-0.395263,0.100499,-0.197590,...,0.758831,-0.457773,-0.304340,0.195167,-0.136565,-0.071396,0.113035,-0.332323,-0.086300,0.294665
7,8,1,5,-0.178741,-0.910932,0.207576,0.592328,-0.103694,-0.484997,0.121915,...,0.290101,-0.383769,0.092301,-0.018747,-0.294603,-0.239061,-0.336887,0.097475,0.124345,0.475675
8,9,1,5,-0.228338,-0.720401,0.171864,0.542521,0.038679,-0.625066,0.004954,...,0.223365,-0.222446,-0.057543,0.050077,-0.153866,-0.229244,-0.310919,0.030602,0.015627,0.419037
9,10,2,9,-0.747268,0.374089,-0.083974,0.947993,-0.419260,0.064127,-0.146026,...,0.866756,0.393436,0.045845,-0.308215,-0.373806,0.235226,-0.261167,-0.265347,-0.974623,0.217985


In [82]:
# df_total= df_total.drop('creative_id', axis=1)
# df_total

,user_id,gender,age,0,1,2,3,4,5,6,...,90,91,92,93,94,95,96,97,98,99
0,1,1,4,-1.273281,0.770723,0.136155,-0.905686,-0.026277,0.210078,1.819711,...,1.006777,0.442474,0.526732,-0.042178,-0.169630,-0.275796,-0.379753,-0.714840,-0.347011,0.134294
1,2,1,10,-0.388902,-0.473549,0.126974,-1.186218,0.057732,0.119775,1.274921,...,0.368814,0.629504,0.246844,0.792099,-0.221441,-0.970539,-0.279602,-0.736155,0.245788,0.103270
2,3,2,7,-0.440877,0.141563,0.311808,-0.891266,0.112476,-0.389019,0.584280,...,0.192744,1.167612,0.409941,0.138162,-0.514415,-0.442151,0.461220,-0.505766,-0.203497,-0.010520
3,4,1,5,-0.468273,0.123332,0.621367,-0.797386,-0.141824,0.303381,0.844034,...,0.582497,0.854060,0.237330,0.297263,0.080774,-0.617994,0.207245,-0.406325,0.312730,-0.067528
4,5,1,4,-0.813797,0.347599,0.602525,-0.245061,0.330083,-0.180306,0.812041,...,0.605953,-0.201676,0.557498,0.335551,0.374775,-0.381907,0.228796,-0.565802,-0.375751,0.063916
5,6,1,6,-0.788292,0.517620,0.379031,-1.247493,-0.177986,-0.402844,0.034946,...,0.266872,0.349065,1.493662,0.211478,0.057508,-0.438076,-0.194376,-1.169862,-0.797381,0.211815
6,7,2,6,-0.417468,-0.192459,-0.416256,-0.931786,0.102557,-1.180209,0.454165,...,0.023957,0.638993,0.781410,0.696175,-0.076834,-0.858763,-0.143843,-0.650961,-0.304911,0.346029
7,8,1,5,-0.528830,0.066346,-0.000931,-0.753874,-0.087022,-0.019050,1.104838,...,0.460458,0.581331,0.528193,0.291288,-0.463458,-0.686376,0.033361,-0.926821,0.664221,-0.121090
8,9,1,5,-0.172013,0.008928,0.323374,-0.593524,-0.324647,0.136153,1.083064,...,0.418885,0.607809,0.097663,0.306342,-0.390083,-0.563756,0.033737,-0.555875,0.542509,0.280699
9,10,2,9,-1.107361,-0.518873,0.160626,-1.015317,0.608227,-1.096342,0.303037,...,1.365656,-0.046279,0.351635,0.644653,0.488358,-1.038564,0.631731,-1.388902,1.745168,0.561204


# 整体处理后区分测试集和训练集

In [ ]:
test = df_total[df_total['gender']==0]
X_test = test.iloc[:,3:]
train = df_total[df_total['gender']!=0]

train['gender'] = train['gender']-1
train['age'] = train['age']-1

# lightgbm方法

## 性别预测

In [10]:
X_train = train.iloc[:,3:]
y_train = train.iloc[:,1]

In [11]:
from sklearn.model_selection import train_test_split

X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.1, random_state=42)

In [43]:
import lightgbm as lgb

print("LGB test")
clf_gender = lgb.LGBMClassifier(boosting_type='gbdt', num_leaves=128, 
    max_depth=10, n_estimators=1000, objective='binary',
    subsample=0.8, colsample_bytree=0.8, subsample_freq=1,
    learning_rate=0.1,   min_child_weight=1, random_state=20, n_jobs=4
    )
clf_gender.fit(X_train, y_train, eval_set=[(X_train, y_train), (X_valid, y_valid)], verbose=10, eval_metric=['acc'], early_stopping_rounds=10)


LGB test
Training until validation scores don't improve for 10 rounds
[10]	training's binary_logloss: 0.363888	valid_1's binary_logloss: 0.366298
[20]	training's binary_logloss: 0.284502	valid_1's binary_logloss: 0.288629
[30]	training's binary_logloss: 0.251926	valid_1's binary_logloss: 0.258149
[40]	training's binary_logloss: 0.235646	valid_1's binary_logloss: 0.243904
[50]	training's binary_logloss: 0.225595	valid_1's binary_logloss: 0.235839
[60]	training's binary_logloss: 0.218534	valid_1's binary_logloss: 0.230829
[70]	training's binary_logloss: 0.212998	valid_1's binary_logloss: 0.227363
[80]	training's binary_logloss: 0.208232	valid_1's binary_logloss: 0.224885
[90]	training's binary_logloss: 0.20397	valid_1's binary_logloss: 0.22275
[100]	training's binary_logloss: 0.200094	valid_1's binary_logloss: 0.221143
[110]	training's binary_logloss: 0.196603	valid_1's binary_logloss: 0.219747
[120]	training's binary_logloss: 0.193376	valid_1's binary_logloss: 0.218581
[130]	training's 

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=0.8,
        importance_type='split', learning_rate=0.1, max_depth=10,
        min_child_samples=20, min_child_weight=1, min_split_gain=0.0,
        n_estimators=1000, n_jobs=4, num_leaves=128, objective='binary',
        random_state=20, reg_alpha=0.0, reg_lambda=0.0, silent=True,
        subsample=0.8, subsample_for_bin=200000, subsample_freq=1)

In [44]:
y_valid_pred=clf_gender.predict(X_valid)
from sklearn.metrics import accuracy_score  
accuracy_score(y_valid, y_valid_pred) 


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


0.9211555555555555

## 年龄预测

In [45]:
X_train = train.iloc[:,3:]
# X_train = pd.concat([X_train, train.iloc[:,1]], axis=1)
print(X_train)
y_train = train.iloc[:,2]
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

        creative_id0  creative_id1  creative_id2  creative_id3  creative_id4  \
0           0.176359      0.003136     -1.137845     -0.319683      0.319190   
1           0.226634     -0.712881     -1.410187     -0.064186      0.128914   
2          -0.052592     -0.108137     -0.811728      0.249566     -0.705926   
3          -0.223217     -0.217504     -0.727354     -0.586052     -0.283344   
4           0.501854     -0.114319     -0.386793      0.016447      0.570212   
5           0.382754      1.408911     -0.471626      0.287415      0.473463   
6           0.169854      0.195228     -0.855291      0.125124      0.165833   
7          -0.171518     -0.073008     -1.030491     -0.549924     -0.202157   
8          -0.337181     -0.201135     -0.830621     -0.502472      0.243263   
9           0.892981      0.267362     -2.413032     -0.411281     -0.862503   
10          0.367931      1.075851     -2.557104     -0.493715      0.154179   
11          0.619828     -0.003680     -

In [46]:
import lightgbm as lgb

print("LGB test")
clf_age = lgb.LGBMClassifier(
        boosting_type='gbdt', num_leaves=55, reg_alpha=0.0, reg_lambda=1,
        max_depth=10, n_estimators=1000, objective='multiclass',
        subsample=0.8, colsample_bytree=0.8, subsample_freq=1,
        learning_rate=0.1,   min_child_weight=1, random_state=20, n_jobs=4
    )
clf_age.fit(X_train, y_train, eval_set=[(X_train, y_train), (X_valid, y_valid)], verbose=10,  early_stopping_rounds=10)


LGB test
Training until validation scores don't improve for 10 rounds
[10]	training's multi_logloss: 1.82117	valid_1's multi_logloss: 1.82904
[20]	training's multi_logloss: 1.72681	valid_1's multi_logloss: 1.74217
[30]	training's multi_logloss: 1.67088	valid_1's multi_logloss: 1.69283
[40]	training's multi_logloss: 1.6326	valid_1's multi_logloss: 1.66075
[50]	training's multi_logloss: 1.60387	valid_1's multi_logloss: 1.6382
[60]	training's multi_logloss: 1.58093	valid_1's multi_logloss: 1.62112
[70]	training's multi_logloss: 1.56179	valid_1's multi_logloss: 1.60778
[80]	training's multi_logloss: 1.54515	valid_1's multi_logloss: 1.59702
[90]	training's multi_logloss: 1.53043	valid_1's multi_logloss: 1.58817
[100]	training's multi_logloss: 1.51705	valid_1's multi_logloss: 1.58049
[110]	training's multi_logloss: 1.50485	valid_1's multi_logloss: 1.57411
[120]	training's multi_logloss: 1.49352	valid_1's multi_logloss: 1.56841
[130]	training's multi_logloss: 1.48283	valid_1's multi_logloss: 

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=0.8,
        importance_type='split', learning_rate=0.1, max_depth=10,
        min_child_samples=20, min_child_weight=1, min_split_gain=0.0,
        n_estimators=1000, n_jobs=4, num_leaves=55, objective='multiclass',
        random_state=20, reg_alpha=0.0, reg_lambda=1, silent=True,
        subsample=0.8, subsample_for_bin=200000, subsample_freq=1)

In [47]:
y_valid_pred=clf_age.predict(X_valid)
from sklearn.metrics import accuracy_score  
accuracy_score(y_valid, y_valid_pred) 

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


0.3828111111111111

In [40]:
gender_pred = clf_gender.predict(X_test)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [45]:
age_pred  = clf_age.predict(X_test)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [46]:
res = test.iloc[:,0:3]
res

,user_id,gender,age
900000,3131989,0,0
900001,3131990,0,0
900002,3131994,0,0
900003,3131995,0,0
900004,3132000,0,0
900005,3132004,0,0
900006,3132007,0,0
900007,3132009,0,0
900008,3132011,0,0
900009,3132012,0,0


In [47]:
res['gender'] = gender_pred
res['age'] = age_pred

In [48]:
res.columns=['user_id', 'predicted_gender','predicted_age']
res = res[['user_id','predicted_age', 'predicted_gender']]
res.to_csv('submission.csv', index=False)

## NN方法

In [ ]:
X_train = train.iloc[:,3:]
y_train = train.iloc[:,1]
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.1, random_state=42)

In [25]:
from keras.layers import Dropout
model = Sequential()
model.add(Dense(256, input_shape=(600,),activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(256, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=3000, batch_size=2048, validation_data=(X_valid, y_valid), callbacks=[early_stopping])

Train on 810000 samples, validate on 90000 samples
Epoch 1/3000
810000/810000 [==============================] - 85s 104us/step - loss: 0.2232 - acc: 0.9143 - val_loss: 0.2068 - val_acc: 0.9219
Epoch 2/3000
810000/810000 [==============================] - 83s 102us/step - loss: 0.2010 - acc: 0.9249 - val_loss: 0.2061 - val_acc: 0.9235TA: 1:05 - loss: 0.2038 - acc: 0.923 - ETA: 1:05 - l - - E - 
Epoch 3/3000
810000/810000 [==============================] - 83s 102us/step - loss: 0.1956 - acc: 0.9275 - val_loss: 0.1958 - val_acc: 0.9272
Epoch 4/3000
810000/810000 [==============================] - 83s 102us/step - loss: 0.1929 - acc: 0.9284 - val_loss: 0.1948 - val_acc: 0.9273.928 - ETA: 6s - loss: 0.1931 - acc: 0.928 - ETA: 5s - loss: 0.1 - ETA: 2s - loss: 0.1931 
Epoch 5/3000
810000/810000 [==============================] - 83s 102us/step - loss: 0.1912 - acc: 0.9289 - val_loss: 0.1927 - val_acc: 0.9285
Epoch 6/3000
810000/810000 [==============================] - 83s 103us/step - loss

In [26]:
clf_gender = model
y_valid_pred=model.predict(X_valid)
y_valid_pred = np.round(y_valid_pred)
accuracy_score(y_valid, y_valid_pred) 

0.9280111111111111

## 年龄预测

In [10]:
X_train = train.iloc[:,3:]
# X_train = pd.concat([X_train, train.iloc[:,1]], axis=1)
print(X_train)
y_train = train.iloc[:,2]
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.1, random_state=42)

        creative_id0  creative_id1  creative_id2  creative_id3  creative_id4  \
0           0.176359      0.003136     -1.137845     -0.319683      0.319190   
1           0.226634     -0.712881     -1.410187     -0.064186      0.128914   
2          -0.052592     -0.108137     -0.811728      0.249566     -0.705926   
3          -0.223217     -0.217504     -0.727354     -0.586052     -0.283344   
4           0.501854     -0.114319     -0.386793      0.016447      0.570212   
5           0.382754      1.408911     -0.471626      0.287415      0.473463   
6           0.169854      0.195228     -0.855291      0.125124      0.165833   
7          -0.171518     -0.073008     -1.030491     -0.549924     -0.202157   
8          -0.337181     -0.201135     -0.830621     -0.502472      0.243263   
9           0.892981      0.267362     -2.413032     -0.411281     -0.862503   
10          0.367931      1.075851     -2.557104     -0.493715      0.154179   
11          0.619828     -0.003680     -

In [ ]:
model = Sequential()
model.add(Dense(256, input_shape=(600,),activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(10, activation='sigmoid'))
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=3000, batch_size=512, validation_data=(X_valid, y_valid), callbacks=[early_stopping])

In [ ]:
clf_age = model
y_valid_pred=model.predict(X_valid)
y_valid_pred = np.argmax(y_valid_pred, axis=1)
accuracy_score(y_valid, y_valid_pred) 

In [32]:
gender_pred = clf_gender.predict(X_test)
age_pred  = clf_age.predict(X_test)

gender_pred = int(np.round(gender_pred))
age_pred = np.argmax(age_pred, axis=1)

gender_pred = gender_pred +1
age_pred = age_pred +1

res = test.iloc[:,0:3]
res['gender'] = gender_pred
res['age'] = age_pred
res.columns=['user_id', 'predicted_gender','predicted_age']
res = res[['user_id','predicted_age', 'predicted_gender']]
res.to_csv('submission.csv', index=False)

In [34]:
res

,user_id,predicted_age,predicted_gender
900000,3131989,6,2.0
900001,3131990,2,2.0
900002,3131994,1,1.0
900003,3131995,5,1.0
900004,3132000,4,2.0
900005,3132004,4,1.0
900006,3132007,2,1.0
900007,3132009,7,1.0
900008,3132011,5,1.0
900009,3132012,3,2.0


## lstm方法

In [49]:
import keras
from keras.layers import Dense
from keras.models import  Sequential
import os
import tensorflow as tf
from keras.callbacks import EarlyStopping

Using TensorFlow backend.


In [50]:
#GPU配置
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
from keras.backend.tensorflow_backend import set_session
config = tf.ConfigProto()
config.gpu_options.allocator_type = 'BFC' #A "Best-fit with coalescing" algorithm, simplified from a version of dlmalloc.
config.gpu_options.per_process_gpu_memory_fraction = 0.6
config.gpu_options.allow_growth = True
set_session(tf.Session(config=config))

#早停定义
early_stopping = EarlyStopping(monitor='val_loss', patience=10, verbose=2)

In [52]:
model = Sequential()
# model.add(keras.layers.CuDNNLSTM(100, input_shape=(300, 1)))
model.add(keras.layers.LSTM(100, input_shape=(700, 1)))
model.add(Dense(10, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 100)               40800     
_________________________________________________________________
dense_1 (Dense)              (None, 10)                1010      
Total params: 41,810
Trainable params: 41,810
Non-trainable params: 0
_________________________________________________________________


In [68]:
y_train = pd.DataFrame(y_train)
y_train = y_train.astype(str)
y_train = pd.get_dummies(y_train)
y_train

,age_1,age_10,age_2,age_3,age_4,age_5,age_6,age_7,age_8,age_9
394657,0,0,0,1,0,0,0,0,0,0
517437,0,0,1,0,0,0,0,0,0,0
208118,0,0,0,0,1,0,0,0,0,0
626437,0,0,0,1,0,0,0,0,0,0
699970,1,0,0,0,0,0,0,0,0,0
691355,0,0,0,0,0,1,0,0,0,0
122754,0,0,0,1,0,0,0,0,0,0
704193,0,0,0,0,0,0,1,0,0,0
721665,0,0,0,0,0,0,1,0,0,0
8676,0,0,0,0,0,0,1,0,0,0


In [69]:
y_train = y_train[['age_1','age_2','age_3','age_4','age_5','age_6','age_7','age_8','age_9','age_10']]

In [70]:
y_train

,age_1,age_2,age_3,age_4,age_5,age_6,age_7,age_8,age_9,age_10
394657,0,0,1,0,0,0,0,0,0,0
517437,0,1,0,0,0,0,0,0,0,0
208118,0,0,0,1,0,0,0,0,0,0
626437,0,0,1,0,0,0,0,0,0,0
699970,1,0,0,0,0,0,0,0,0,0
691355,0,0,0,0,1,0,0,0,0,0
122754,0,0,1,0,0,0,0,0,0,0
704193,0,0,0,0,0,1,0,0,0,0
721665,0,0,0,0,0,1,0,0,0,0
8676,0,0,0,0,0,1,0,0,0,0


In [71]:
y_valid = pd.DataFrame(y_valid)
y_valid = y_valid.astype(str)
y_valid = pd.get_dummies(y_valid)
y_valid = y_valid[['age_1','age_2','age_3','age_4','age_5','age_6','age_7','age_8','age_9','age_10']]
y_valid

,age_1,age_2,age_3,age_4,age_5,age_6,age_7,age_8,age_9,age_10
527984,1,0,0,0,0,0,0,0,0,0
618093,0,0,0,0,0,0,1,0,0,0
484217,0,0,0,0,1,0,0,0,0,0
161123,0,0,0,0,0,1,0,0,0,0
718642,0,0,1,0,0,0,0,0,0,0
234835,0,1,0,0,0,0,0,0,0,0
484013,0,0,1,0,0,0,0,0,0,0
555953,0,1,0,0,0,0,0,0,0,0
282983,0,0,1,0,0,0,0,0,0,0
619021,0,0,0,0,0,0,0,0,1,0


In [72]:
X_valid

,creative_id0_x,creative_id1_x,creative_id2_x,creative_id3_x,creative_id4_x,creative_id5_x,creative_id6_x,creative_id7_x,creative_id8_x,creative_id9_x,...,product_id90,product_id91,product_id92,product_id93,product_id94,product_id95,product_id96,product_id97,product_id98,product_id99
527984,0.547468,0.039332,-1.149888,0.089806,-0.173504,-0.057703,-1.461289,-0.025330,0.933001,-0.158257,...,-0.011304,0.464129,0.209207,0.198498,0.498906,0.343027,-0.383254,-0.029144,-0.393500,-0.252587
618093,-0.482383,-0.384994,-0.997014,-0.370805,-0.222202,0.233154,-0.956954,-1.306384,1.096096,-0.522591,...,0.481019,0.089266,0.181807,0.225739,0.309112,0.000860,0.027332,0.372650,0.254558,-0.193108
484217,0.502271,0.711300,-1.267665,0.409578,0.015751,1.253626,-1.382575,-0.889556,1.292730,-1.688393,...,0.421600,0.257555,0.718687,0.425558,0.356124,0.321190,-0.652347,0.138614,0.154275,0.233896
161123,-0.011356,0.995864,-1.650180,-0.097341,-0.436036,-0.124533,-1.210603,-0.175618,1.442178,-0.635773,...,0.363485,0.419983,0.115620,0.827169,0.174141,-0.022154,-0.258182,-0.178880,-0.034826,0.343419
718642,0.677317,0.763415,-0.791646,-0.333481,0.714576,0.152541,-0.708335,-0.465027,0.763585,-0.413722,...,0.647645,0.482548,-0.598005,0.284750,0.524364,0.457291,0.260921,0.660853,0.641223,0.840390
234835,0.184396,-0.260994,-2.007082,-0.357509,-0.389807,0.846841,-1.785337,-0.080217,1.480832,-1.087203,...,0.455838,-0.449918,0.151940,0.986617,1.372821,-0.122037,-1.008023,1.083719,-0.619470,-0.413207
484013,-0.282848,0.550064,-1.900675,-0.709062,-0.565912,-0.192109,-1.034285,-0.089343,0.566467,-0.875182,...,0.408905,0.005295,0.318825,0.743797,0.221327,0.493759,-0.681392,-0.233765,0.299105,0.208343
555953,-0.954310,0.148639,-1.059264,-0.585269,0.732900,0.022107,-0.795439,-0.625391,1.027700,0.194162,...,0.068478,0.258028,0.474217,0.427147,0.490252,0.462752,-0.077236,0.531048,-0.335576,-0.460838
282983,0.566963,0.389838,-0.660934,-0.236560,0.551830,0.931780,-2.055758,-0.779672,0.106700,-0.296297,...,0.085265,-0.591826,0.794633,0.579417,0.715664,0.137131,-0.709600,0.590262,-0.083568,-0.054560
619021,-0.214907,-0.122564,-1.264682,-0.547263,-0.135129,0.470418,-1.648296,-0.359655,1.631953,-0.379480,...,0.585079,0.523780,-0.501520,0.536121,0.866123,-0.453387,-0.168415,0.993594,-0.556592,-0.108511


In [73]:
X_train

,creative_id0_x,creative_id1_x,creative_id2_x,creative_id3_x,creative_id4_x,creative_id5_x,creative_id6_x,creative_id7_x,creative_id8_x,creative_id9_x,...,product_id90,product_id91,product_id92,product_id93,product_id94,product_id95,product_id96,product_id97,product_id98,product_id99
394657,-1.207838,-0.338209,-1.482748,-1.182228,-0.457005,0.391338,-1.347567,0.078981,0.135401,-0.181177,...,0.634890,-0.302710,0.837662,-0.105968,1.202575,0.112359,-0.374152,1.147488,0.167767,-0.508857
517437,0.130191,-0.738891,-1.759662,-0.483882,-1.349669,0.685947,-1.399326,-0.378762,0.990070,-1.419389,...,0.390362,-0.397495,0.175349,1.158787,1.197022,-0.253866,-0.906495,1.101163,-0.557697,-0.703362
208118,0.215777,0.331387,-0.758553,-0.067916,-0.355351,0.129438,-1.300004,0.178100,0.948770,-0.579030,...,0.920531,-0.466015,-0.513684,0.061684,0.372448,-0.513134,0.075694,1.090132,-0.018660,0.611876
626437,0.018725,-0.100871,-0.899868,-1.075308,0.258474,0.360591,-0.640741,-0.165778,0.545021,-0.580780,...,-0.040487,0.362969,0.091402,0.906636,0.611700,0.519764,-0.749374,-0.178186,0.215880,0.388665
699970,-0.529019,-0.114842,-0.970519,0.188741,-0.845489,-0.323962,-1.120515,0.075817,0.788997,0.397173,...,0.294885,0.784469,-0.186788,-0.073041,0.353242,-0.057510,0.468490,0.717139,-0.710294,-1.093519
691355,0.030283,0.257035,-1.489001,-0.320903,-0.335252,-0.217358,-0.341236,0.342869,0.687184,-0.196522,...,0.232987,-0.091291,0.681323,0.981857,0.558979,0.444527,-0.432819,0.320552,-0.331205,0.271193
122754,0.434407,0.132300,-1.172779,-0.691829,0.745311,0.618859,-1.082027,0.452470,0.494951,-0.829951,...,0.026679,0.241151,0.420142,0.683039,0.678975,0.703925,-0.373010,0.101212,0.105470,0.380592
704193,0.143477,-0.032145,-0.138412,0.544023,0.696715,0.063063,-0.940159,-0.198911,0.609021,0.185299,...,0.801061,0.353214,0.761210,-0.278068,0.168770,-0.356377,-0.314367,1.896161,0.056070,-0.215113
721665,0.398260,-0.129157,-0.356981,-0.016210,0.317492,0.326673,-0.745034,-0.329013,0.656436,0.358443,...,0.816981,0.494375,0.422627,0.171711,0.321002,-0.307651,0.033722,1.015686,0.146280,-0.171041
8676,1.272892,0.571621,-2.094541,-0.339541,0.576456,0.037051,-0.919126,0.695130,0.986774,-0.891953,...,0.193772,0.029583,0.776270,0.775509,0.486168,0.478126,-0.566177,0.490157,-0.112621,-0.059686


In [74]:
X_train = np.array(X_train).reshape(-1, 700, 1)
X_valid = np.array(X_valid).reshape(-1, 700, 1)

In [75]:
y_train = np.array(y_train)
y_valid = np.array(y_valid)

In [76]:
X_train.shape, y_train.shape, X_valid.shape, y_valid.shape

((720000, 700, 1), (720000, 10), (180000, 700, 1), (180000, 10))

In [5]:
# np.save('X_train.npy', X_train)
# np.save('y_train.npy', y_train)
# np.save('X_valid.npy', X_valid)
# np.save('y_valid.npy', y_valid)

# X_train = np.load('X_train.npy')
# y_train = np.load('y_train.npy')
# X_valid = np.load('X_valid.npy')
# y_valid = np.load('y_valid.npy')

In [ ]:
model.fit(X_train, y_train, epochs=3000, batch_size=2048, validation_data=(X_valid, y_valid), callbacks=[early_stopping])

Train on 720000 samples, validate on 180000 samples
Epoch 1/3000
720000/720000 [==============================] - 5913s 8ms/step - loss: 2.0203 - acc: 0.2239 - val_loss: 1.9731 - val_acc: 0.2255
Epoch 2/3000
720000/720000 [==============================] - 5918s 8ms/step - loss: 1.9513 - acc: 0.2345 - val_loss: 1.9375 - val_acc: 0.2372
Epoch 3/3000
720000/720000 [==============================] - 5917s 8ms/step - loss: 1.9242 - acc: 0.2435 - val_loss: 1.9265 - val_acc: 0.2433
Epoch 4/3000
720000/720000 [==============================] - 5914s 8ms/step - loss: 1.9097 - acc: 0.2484 - val_loss: 1.9056 - val_acc: 0.2489
Epoch 5/3000
720000/720000 [==============================] - 5916s 8ms/step - loss: 1.8984 - acc: 0.2514 - val_loss: 1.8901 - val_acc: 0.2519
Epoch 6/3000
718848/720000 [============================>.] - ETA: 8s - loss: 1.8908 - acc: 0.2533 